# 📚 Vietnamese Text Summarization - Extractive & Abstractive

## Tổng quan dự án

**Mục tiêu**: So sánh 2 phương pháp tóm tắt văn bản tiếng Việt
- **Extractive**: Chọn lọc câu có sẵn (PhoBERT + TextRank)
- **Abstractive**: Sinh câu mới hoàn toàn (ViT5)

**Dataset**: VLSP 2021 Summarization Task

---

## 📋 Nội dung

1. **Lý thuyết Text Summarization**
2. **Setup & Load Data**
3. **Extractive Summarization**
   - TextRank Algorithm
   - PhoBERT Sentence Scoring
4. **Abstractive Summarization**
   - ViT5 Fine-tuning
   - Generation Strategies
5. **Evaluation & Comparison**

---

## 1️⃣ LÝ THUYẾT: EXTRACTIVE vs ABSTRACTIVE

### A. Extractive Summarization 📋

**Định nghĩa**: Chọn lọc các câu quan trọng từ văn bản gốc

```
Input (3 câu):
  [1] "Việt Nam là một đất nước đông dân ở Đông Nam Á."
  [2] "Nền kinh tế phát triển nhanh với tốc độ tăng trưởng cao."
  [3] "Văn hóa đa dạng với 54 dân tộc."

Extractive Output:
  ↓ Chọn câu [1] và [2] (score cao nhất)
  "Việt Nam là một đất nước đông dân ở Đông Nam Á. Nền kinh tế phát triển nhanh."
```

**Phương pháp:**

1. **TextRank** (Graph-based)
   - Xây dựng graph với nodes = câu
   - Edges = độ tương đồng giữa các câu
   - Chạy PageRank algorithm
   - Chọn top-k câu có score cao
**Ưu điểm:**
- ✅ Ngữ pháp chuẩn (dùng câu gốc)
- ✅ Không tạo thông tin sai (no hallucination)
- ✅ Training đơn giản, ít resource
- ✅ Nhanh, dễ deploy

**Nhược điểm:**
- ❌ Tính mạch lạc không cao
- ❌ Không thể paraphrase
- ❌ Kém linh hoạt về độ dài
- ❌ Có thể chứa thông tin thừa

---

### B. Abstractive Summarization 🤖

**Định nghĩa**: Sinh câu mới hoàn toàn, giống cách con người tóm tắt

```
Input (3 câu):
  "Việt Nam là một đất nước đông dân ở Đông Nam Á."
  "Nền kinh tế phát triển nhanh với tốc độ tăng trưởng cao."
  "Văn hóa đa dạng với 54 dân tộc."

Abstractive Output:
  ↓ Tạo câu mới, tổng hợp thông tin
  "Việt Nam có dân số đông, nền kinh tế tăng trưởng nhanh và văn hóa đa dạng."
```

**Phương pháp:**

**ViT5** (Vietnamese T5)
- Encoder-Decoder Transformer
- Pre-trained trên Vietnamese corpus
- Generation với Beam Search/Sampling

```
Document → [Encoder] → Context → [Decoder] → Summary
             ViT5-enc             ViT5-dec
                        ↓
                 Attention weights
```

**Ưu điểm:**
- ✅ Tự nhiên, mạch lạc như người viết
- ✅ Có thể paraphrase, tổng hợp thông tin
- ✅ Linh hoạt về độ dài và style
- ✅ Loại bỏ thông tin thừa tốt hơn

**Nhược điểm:**
- ❌ Dễ gặp lỗi logic, hallucination
- ❌ Cần nhiều tài nguyên training
- ❌ Phức tạp, khó debug
- ❌ Ngữ pháp có thể sai

---

### C. So sánh trực quan

| Tiêu chí | Extractive | Abstractive |
|----------|------------|-------------|
| **Ngữ pháp** | ✅ Luôn đúng | ⚠️ Có thể sai |
| **Mạch lạc** | ⚠️ Trung bình | ✅ Tốt |
| **Tự nhiên** | ⚠️ Cứng nhắc | ✅ Như người viết |
| **Hallucination** | ✅ Không có | ❌ Có thể xảy ra |
| **Training** | ✅ Đơn giản | ❌ Phức tạp |
| **Speed** | ✅ Nhanh | ⚠️ Chậm hơn |
| **Flexibility** | ❌ Thấp | ✅ Cao |

**Kết luận:**
- **Extractive**: Tốt cho tin tức, báo cáo, cần độ chính xác cao
- **Abstractive**: Tốt cho tóm tắt dài, content marketing, creative writing

---

## 2️⃣ SETUP ENVIRONMENT

In [ ]:
# ============================================================================
# Install Required Packages
# ============================================================================
!pip install -q transformers>=4.50.0 accelerate>=0.26.0 datasets>=2.14.6
!pip install -q sentencepiece>=0.1.99 rouge-score>=0.1.2 evaluate>=0.4.1
!pip install -q torch torchvision torchaudio
!pip install -q underthesea  # For Vietnamese text processing
!pip install -q scikit-learn networkx  # For TextRank

print("✅ All packages installed!")

In [ ]:
# ============================================================================
# Import Libraries
# ============================================================================
import os
import re
import numpy as np
import pandas as pd
import torch
import warnings
from typing import List, Dict, Tuple
from tqdm.auto import tqdm

# Text processing
from underthesea import sent_tokenize

# Transformers
from transformers import (
    AutoTokenizer, 
    AutoModel,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)

# Evaluation
from rouge_score import rouge_scorer
import evaluate

# Graph algorithms
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 3️⃣ LOAD & EXPLORE DATA

In [ ]:
# ============================================================================
# Load VLSP 2021 Dataset
# ============================================================================
from datasets import load_dataset

print("Loading VLSP 2021 Summarization dataset...")

# Load dataset
dataset = load_dataset("Linh2703/VLSP2021_summarization")

# Convert to pandas for easier manipulation
train_df = pd.DataFrame(dataset['train'])
val_df = pd.DataFrame(dataset['validation'])
test_df = pd.DataFrame(dataset['test'])

print(f"\n📊 Dataset Statistics:")
print(f"  Train: {len(train_df):,} samples")
print(f"  Validation: {len(val_df):,} samples")
print(f"  Test: {len(test_df):,} samples")

# Show sample
print(f"\n📄 Sample data:")
sample = train_df.iloc[0]
print(f"\nDocument (first 200 chars):\n{sample['document'][:200]}...")
print(f"\nSummary:\n{sample['summary']}")

In [ ]:
# ============================================================================
# Data Statistics
# ============================================================================
def analyze_text_lengths(df: pd.DataFrame, name: str):
    """Analyze document and summary lengths"""
    doc_words = df['document'].apply(lambda x: len(x.split()))
    sum_words = df['summary'].apply(lambda x: len(x.split()))
    
    doc_sents = df['document'].apply(lambda x: len(sent_tokenize(x)))
    sum_sents = df['summary'].apply(lambda x: len(sent_tokenize(x)))
    
    print(f"\n{name} Statistics:")
    print(f"  Document words: mean={doc_words.mean():.1f}, median={doc_words.median():.1f}")
    print(f"  Summary words: mean={sum_words.mean():.1f}, median={sum_words.median():.1f}")
    print(f"  Document sentences: mean={doc_sents.mean():.1f}, median={doc_sents.median():.1f}")
    print(f"  Summary sentences: mean={sum_sents.mean():.1f}, median={sum_sents.median():.1f}")
    print(f"  Compression ratio: {(sum_words.mean() / doc_words.mean() * 100):.1f}%")

analyze_text_lengths(train_df, "Train")
analyze_text_lengths(val_df, "Validation")
analyze_text_lengths(test_df, "Test")

## 4️⃣ EXTRACTIVE SUMMARIZATION

### Method 1: TextRank (Graph-based)

In [ ]:
# ============================================================================
# TextRank Implementation
# ============================================================================
class TextRankSummarizer:
    """TextRank algorithm for extractive summarization"""
    
    def __init__(self, top_n: int = 3, damping: float = 0.85):
        self.top_n = top_n
        self.damping = damping
        self.tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')
        self.model = AutoModel.from_pretrained('vinai/phobert-base')
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
    
    def get_sentence_embedding(self, sentence: str) -> np.ndarray:
        """Get PhoBERT embedding for a sentence"""
        inputs = self.tokenizer(
            sentence, 
            return_tensors='pt', 
            truncation=True, 
            max_length=256
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
            # Use CLS token embedding
            embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        
        return embedding[0]
    
    def build_similarity_matrix(self, sentences: List[str]) -> np.ndarray:
        """Build similarity matrix between sentences"""
        print(f"  Computing embeddings for {len(sentences)} sentences...")
        embeddings = []
        
        for sent in tqdm(sentences, desc="Encoding"):
            emb = self.get_sentence_embedding(sent)
            embeddings.append(emb)
        
        embeddings = np.array(embeddings)
        similarity_matrix = cosine_similarity(embeddings)
        
        return similarity_matrix
    
    def textrank(self, similarity_matrix: np.ndarray) -> np.ndarray:
        """Run TextRank algorithm (PageRank on sentence graph)"""
        # Create graph from similarity matrix
        nx_graph = nx.from_numpy_array(similarity_matrix)
        
        # Compute PageRank scores
        scores = nx.pagerank(nx_graph, alpha=self.damping)
        
        return np.array(list(scores.values()))
    
    def summarize(self, document: str, num_sentences: int = None) -> str:
        """Generate extractive summary using TextRank"""
        if num_sentences is None:
            num_sentences = self.top_n
        
        # Split into sentences
        sentences = sent_tokenize(document)
        
        if len(sentences) <= num_sentences:
            return document
        
        # Build similarity matrix
        similarity_matrix = self.build_similarity_matrix(sentences)
        
        # Run TextRank
        scores = self.textrank(similarity_matrix)
        
        # Select top sentences
        ranked_indices = np.argsort(scores)[::-1][:num_sentences]
        
        # Sort by original order to maintain coherence
        ranked_indices = sorted(ranked_indices)
        
        # Extract summary
        summary_sentences = [sentences[i] for i in ranked_indices]
        summary = ' '.join(summary_sentences)
        
        return summary

print("✅ TextRank Summarizer created!")

### Method 2: PhoBERT Sentence Scoring

In [ ]:
# ============================================================================
# PhoBERT Sentence Scorer
# ============================================================================
class PhoBERTExtractor:
    """Extractive summarization using PhoBERT sentence scoring"""
    
    def __init__(self, top_n: int = 3):
        self.top_n = top_n
        self.tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')
        self.model = AutoModel.from_pretrained('vinai/phobert-base')
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
    
    def get_embedding(self, text: str) -> np.ndarray:
        """Get PhoBERT embedding"""
        inputs = self.tokenizer(
            text, 
            return_tensors='pt', 
            truncation=True, 
            max_length=256
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
            embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        
        return embedding[0]
    
    def score_sentences(self, document: str) -> List[Tuple[int, float, str]]:
        """Score sentences by similarity to document"""
        # Split into sentences
        sentences = sent_tokenize(document)
        
        # Get document embedding
        doc_embedding = self.get_embedding(document)
        
        # Score each sentence
        scores = []
        for idx, sent in enumerate(sentences):
            sent_embedding = self.get_embedding(sent)
            
            # Cosine similarity
            similarity = cosine_similarity(
                doc_embedding.reshape(1, -1),
                sent_embedding.reshape(1, -1)
            )[0][0]
            
            # Add position bias (earlier sentences are more important)
            position_weight = 1.0 - (idx / len(sentences)) * 0.3
            final_score = similarity * position_weight
            
            scores.append((idx, final_score, sent))
        
        return scores
    
    def summarize(self, document: str, num_sentences: int = None) -> str:
        """Generate extractive summary"""
        if num_sentences is None:
            num_sentences = self.top_n
        
        sentences = sent_tokenize(document)
        
        if len(sentences) <= num_sentences:
            return document
        
        # Score sentences
        scores = self.score_sentences(document)
        
        # Sort by score and select top
        top_sentences = sorted(scores, key=lambda x: x[1], reverse=True)[:num_sentences]
        
        # Sort by original position
        top_sentences = sorted(top_sentences, key=lambda x: x[0])
        
        # Extract summary
        summary = ' '.join([sent[2] for sent in top_sentences])
        
        return summary

print("✅ PhoBERT Extractor created!")

### Test Extractive Methods

In [ ]:
# ============================================================================
# Test Extractive Summarization
# ============================================================================
# Sample document
sample_doc = train_df.iloc[0]['document']
sample_ref = train_df.iloc[0]['summary']

print("="*80)
print("EXTRACTIVE SUMMARIZATION TEST")
print("="*80)

print("\n📄 Original Document:")
print(sample_doc[:500] + "...")

print("\n📝 Reference Summary:")
print(sample_ref)

# Method 1: PhoBERT Extractor (faster)
print("\n" + "="*80)
print("Method 1: PhoBERT Sentence Scoring")
print("="*80)
phobert_extractor = PhoBERTExtractor(top_n=3)
phobert_summary = phobert_extractor.summarize(sample_doc)
print("\n✅ PhoBERT Summary:")
print(phobert_summary)

# Method 2: TextRank (more sophisticated but slower)
# Uncomment if you want to test TextRank
# print("\n" + "="*80)
# print("Method 2: TextRank Algorithm")
# print("="*80)
# textrank_summarizer = TextRankSummarizer(top_n=3)
# textrank_summary = textrank_summarizer.summarize(sample_doc)
# print("\n✅ TextRank Summary:")
# print(textrank_summary)

## 5️⃣ ABSTRACTIVE SUMMARIZATION WITH ViT5

In [ ]:
# ============================================================================
# ViT5 Configuration
# ============================================================================
VIT5_MODEL_NAME = 'VietAI/vit5-base'
MAX_LENGTH = 512
MAX_TARGET_LENGTH = 128
BATCH_SIZE = 4
LEARNING_RATE = 5e-5
NUM_EPOCHS = 3

print("="*80)
print("ViT5 MODEL CONFIGURATION")
print("="*80)
print(f"Model: {VIT5_MODEL_NAME}")
print(f"Max input length: {MAX_LENGTH}")
print(f"Max output length: {MAX_TARGET_LENGTH}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Epochs: {NUM_EPOCHS}")
print("="*80)

In [ ]:
# ============================================================================
# Load ViT5 Model & Tokenizer
# ============================================================================
print("Loading ViT5 model...")

vit5_tokenizer = AutoTokenizer.from_pretrained(VIT5_MODEL_NAME)
vit5_model = AutoModelForSeq2SeqLM.from_pretrained(VIT5_MODEL_NAME)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vit5_model.to(device)

print(f"\n✅ ViT5 Model loaded successfully!")
print(f"  Device: {device}")
print(f"  Parameters: {sum(p.numel() for p in vit5_model.parameters()):,}")

In [ ]:
# ============================================================================
# Prepare Dataset for ViT5
# ============================================================================
def preprocess_function(examples):
    """Tokenize documents and summaries"""
    inputs = ["summarize: " + doc for doc in examples['document']]
    
    model_inputs = vit5_tokenizer(
        inputs,
        max_length=MAX_LENGTH,
        truncation=True,
        padding='max_length'
    )
    
    # Tokenize targets
    labels = vit5_tokenizer(
        examples['summary'],
        max_length=MAX_TARGET_LENGTH,
        truncation=True,
        padding='max_length'
    )
    
    model_inputs['labels'] = labels['input_ids']
    
    return model_inputs

print("Preparing datasets...")

# Tokenize datasets
tokenized_train = dataset['train'].map(
    preprocess_function,
    batched=True,
    remove_columns=['document', 'summary']
)

tokenized_val = dataset['validation'].map(
    preprocess_function,
    batched=True,
    remove_columns=['document', 'summary']
)

print(f"✅ Datasets prepared!")
print(f"  Train: {len(tokenized_train)} samples")
print(f"  Validation: {len(tokenized_val)} samples")

In [ ]:
# ============================================================================
# Training Arguments & Trainer
# ============================================================================
rouge_metric = evaluate.load('rouge')

def compute_metrics(eval_pred):
    """Compute ROUGE metrics"""
    predictions, labels = eval_pred
    
    # Decode predictions
    decoded_preds = vit5_tokenizer.batch_decode(
        predictions, 
        skip_special_tokens=True
    )
    
    # Replace -100 in labels (padding)
    labels = np.where(labels != -100, labels, vit5_tokenizer.pad_token_id)
    decoded_labels = vit5_tokenizer.batch_decode(
        labels, 
        skip_special_tokens=True
    )
    
    # Compute ROUGE
    result = rouge_metric.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )
    
    return {
        'rouge1': round(result['rouge1'] * 100, 2),
        'rouge2': round(result['rouge2'] * 100, 2),
        'rougeL': round(result['rougeL'] * 100, 2)
    }

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./vit5_summarization",
    overwrite_output_dir=True,
    
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=3,
    
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    greater_is_better=True,
    
    learning_rate=LEARNING_RATE,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    
    gradient_accumulation_steps=2,
    warmup_steps=500,
    weight_decay=0.01,
    
    fp16=torch.cuda.is_available(),
    
    logging_steps=100,
    report_to="none",
    
    predict_with_generate=True,
    generation_max_length=MAX_TARGET_LENGTH,
    generation_num_beams=4,
    
    seed=42,
)

# Data collator
data_collator = DataCollatorForSeq2Seq(
    vit5_tokenizer,
    model=vit5_model,
    padding=True
)

# Create trainer
trainer = Seq2SeqTrainer(
    model=vit5_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=vit5_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("✅ ViT5 Trainer initialized!")
print("\n⚠️  To train: trainer.train()")
print("⚠️  This will take several hours!")

In [ ]:
# ============================================================================
# Train ViT5 Model (OPTIONAL - takes hours)
# ============================================================================
# Uncomment to train
# print("Starting training...")
# trainer.train()
# print("✅ Training completed!")

# Save model
# trainer.save_model("./vit5_final_model")
# vit5_tokenizer.save_pretrained("./vit5_final_model")

In [ ]:
# ============================================================================
# ViT5 Inference Function
# ============================================================================
def generate_abstractive_summary(
    document: str,
    max_length: int = 128,
    num_beams: int = 4,
    temperature: float = 1.0,
    top_p: float = 0.9
) -> str:
    """Generate abstractive summary using ViT5"""
    
    # Prepare input
    input_text = f"summarize: {document}"
    inputs = vit5_tokenizer(
        input_text,
        max_length=MAX_LENGTH,
        truncation=True,
        return_tensors='pt'
    ).to(device)
    
    # Generate
    with torch.no_grad():
        outputs = vit5_model.generate(
            **inputs,
            max_length=max_length,
            num_beams=num_beams,
            temperature=temperature,
            top_p=top_p,
            early_stopping=True,
            no_repeat_ngram_size=3,
            length_penalty=1.0
        )
    
    # Decode
    summary = vit5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return summary

print("✅ ViT5 inference function ready!")

In [ ]:
# ============================================================================
# Test Abstractive Summarization
# ============================================================================
sample_doc = train_df.iloc[0]['document']
sample_ref = train_df.iloc[0]['summary']

print("="*80)
print("ABSTRACTIVE SUMMARIZATION TEST")
print("="*80)

print("\n📄 Original Document:")
print(sample_doc[:500] + "...")

print("\n📝 Reference Summary:")
print(sample_ref)

print("\n" + "="*80)
print("ViT5 Generation Strategies")
print("="*80)

# Strategy 1: Beam Search (most common)
print("\n1️⃣ Beam Search (num_beams=4):")
beam_summary = generate_abstractive_summary(
    sample_doc,
    num_beams=4,
    temperature=1.0
)
print(beam_summary)

# Strategy 2: Sampling with temperature
print("\n2️⃣ Sampling (temperature=0.7, top_p=0.9):")
sampling_summary = generate_abstractive_summary(
    sample_doc,
    num_beams=1,
    temperature=0.7,
    top_p=0.9
)
print(sampling_summary)

# Strategy 3: Greedy (deterministic)
print("\n3️⃣ Greedy Decoding (num_beams=1):")
greedy_summary = generate_abstractive_summary(
    sample_doc,
    num_beams=1,
    temperature=1.0
)
print(greedy_summary)

## 6️⃣ EVALUATION & COMPARISON

In [ ]:
# ============================================================================
# ROUGE Evaluation Function
# ============================================================================
def evaluate_rouge(predictions: List[str], references: List[str]) -> Dict:
    """Calculate ROUGE scores"""
    scorer = rouge_scorer.RougeScorer(
        ['rouge1', 'rouge2', 'rougeL'],
        use_stemmer=True
    )
    
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    
    for pred, ref in zip(predictions, references):
        scores = scorer.score(ref, pred)
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)
    
    return {
        'rouge1': np.mean(rouge1_scores) * 100,
        'rouge2': np.mean(rouge2_scores) * 100,
        'rougeL': np.mean(rougeL_scores) * 100
    }

print("✅ Evaluation function ready!")

In [ ]:
# ============================================================================
# Compare Extractive vs Abstractive on Test Set
# ============================================================================
print("="*80)
print("EVALUATING ON TEST SET")
print("="*80)

# Use subset for quick evaluation
test_subset = test_df.head(50)  # Adjust size as needed

print(f"\nEvaluating {len(test_subset)} samples...\n")

# Initialize summarizers
phobert_extractor = PhoBERTExtractor(top_n=3)

# Store results
extractive_summaries = []
abstractive_summaries = []
references = test_subset['summary'].tolist()

# Generate summaries
for idx, row in tqdm(test_subset.iterrows(), total=len(test_subset), desc="Generating summaries"):
    doc = row['document']
    
    # Extractive
    ext_summary = phobert_extractor.summarize(doc)
    extractive_summaries.append(ext_summary)
    
    # Abstractive
    abs_summary = generate_abstractive_summary(doc)
    abstractive_summaries.append(abs_summary)

# Evaluate
print("\n" + "="*80)
print("ROUGE SCORES COMPARISON")
print("="*80)

extractive_scores = evaluate_rouge(extractive_summaries, references)
abstractive_scores = evaluate_rouge(abstractive_summaries, references)

print("\n📊 Extractive Summarization (PhoBERT):")
print(f"  ROUGE-1: {extractive_scores['rouge1']:.2f}%")
print(f"  ROUGE-2: {extractive_scores['rouge2']:.2f}%")
print(f"  ROUGE-L: {extractive_scores['rougeL']:.2f}%")

print("\n🤖 Abstractive Summarization (ViT5):")
print(f"  ROUGE-1: {abstractive_scores['rouge1']:.2f}%")
print(f"  ROUGE-2: {abstractive_scores['rouge2']:.2f}%")
print(f"  ROUGE-L: {abstractive_scores['rougeL']:.2f}%")

# Compare
print("\n" + "="*80)
print("PERFORMANCE COMPARISON")
print("="*80)

r1_diff = abstractive_scores['rouge1'] - extractive_scores['rouge1']
r2_diff = abstractive_scores['rouge2'] - extractive_scores['rouge2']
rL_diff = abstractive_scores['rougeL'] - extractive_scores['rougeL']

print(f"\nROUGE-1 difference: {r1_diff:+.2f}% {'(Abstractive wins)' if r1_diff > 0 else '(Extractive wins)'}")
print(f"ROUGE-2 difference: {r2_diff:+.2f}% {'(Abstractive wins)' if r2_diff > 0 else '(Extractive wins)'}")
print(f"ROUGE-L difference: {rL_diff:+.2f}% {'(Abstractive wins)' if rL_diff > 0 else '(Extractive wins)'}")

In [ ]:
# ============================================================================
# Qualitative Comparison
# ============================================================================
print("="*80)
print("QUALITATIVE ANALYSIS - Side by Side Comparison")
print("="*80)

# Show 3 examples
for i in range(min(3, len(test_subset))):
    print(f"\n{'='*80}")
    print(f"Example {i+1}")
    print("="*80)
    
    doc = test_subset.iloc[i]['document']
    ref = test_subset.iloc[i]['summary']
    ext = extractive_summaries[i]
    abs_ = abstractive_summaries[i]
    
    print(f"\n📄 Document (first 300 chars):\n{doc[:300]}...\n")
    
    print(f"📝 Reference Summary:\n{ref}\n")
    
    print(f"📋 Extractive Summary (PhoBERT):\n{ext}\n")
    
    print(f"🤖 Abstractive Summary (ViT5):\n{abs_}\n")
    
    # Compare characteristics
    print(f"📊 Statistics:")
    print(f"  Reference: {len(ref.split())} words, {len(sent_tokenize(ref))} sentences")
    print(f"  Extractive: {len(ext.split())} words, {len(sent_tokenize(ext))} sentences")
    print(f"  Abstractive: {len(abs_.split())} words, {len(sent_tokenize(abs_))} sentences")

## 7️⃣ FINAL ANALYSIS & INSIGHTS

In [ ]:
# ============================================================================
# Summary Statistics
# ============================================================================
print("="*80)
print("FINAL ANALYSIS: EXTRACTIVE vs ABSTRACTIVE")
print("="*80)

# Length analysis
ext_lengths = [len(s.split()) for s in extractive_summaries]
abs_lengths = [len(s.split()) for s in abstractive_summaries]
ref_lengths = [len(s.split()) for s in references]

print("\n📏 Summary Length Analysis:")
print(f"  Reference: {np.mean(ref_lengths):.1f} ± {np.std(ref_lengths):.1f} words")
print(f"  Extractive: {np.mean(ext_lengths):.1f} ± {np.std(ext_lengths):.1f} words")
print(f"  Abstractive: {np.mean(abs_lengths):.1f} ± {np.std(abs_lengths):.1f} words")

# Sentence count
ext_sents = [len(sent_tokenize(s)) for s in extractive_summaries]
abs_sents = [len(sent_tokenize(s)) for s in abstractive_summaries]
ref_sents = [len(sent_tokenize(s)) for s in references]

print("\n📝 Sentence Count:")
print(f"  Reference: {np.mean(ref_sents):.1f} sentences")
print(f"  Extractive: {np.mean(ext_sents):.1f} sentences")
print(f"  Abstractive: {np.mean(abs_sents):.1f} sentences")

# Final recommendations
print("\n" + "="*80)
print("KEY FINDINGS & RECOMMENDATIONS")
print("="*80)

print("""
1️⃣ EXTRACTIVE SUMMARIZATION:
   ✅ Ưu điểm:
      - Ngữ pháp luôn chính xác (dùng câu gốc)
      - Không tạo thông tin sai
      - Training đơn giản, deploy dễ dàng
      - Tốc độ nhanh
   
   ❌ Nhược điểm:
      - Kém tự nhiên, có thể thiếu mạch lạc
      - Không thể tổng hợp thông tin
      - Độ dài khó kiểm soát
   
   💡 Phù hợp cho:
      - Tin tức, báo cáo chính thức
      - Tài liệu kỹ thuật
      - Khi cần độ chính xác cao

2️⃣ ABSTRACTIVE SUMMARIZATION:
   ✅ Ưu điểm:
      - Tự nhiên, mạch lạc
      - Có thể tổng hợp và paraphrase
      - Linh hoạt về độ dài
      - Quality cao hơn khi fine-tuned tốt
   
   ❌ Nhược điểm:
      - Có thể tạo thông tin sai (hallucination)
      - Cần nhiều resource để train
      - Phức tạp hơn
   
   💡 Phù hợp cho:
      - Content marketing
      - Tóm tắt dài, creative writing
      - Khi cần summary tự nhiên

3️⃣ HYBRID APPROACH (Khuyến nghị):
   💡 Kết hợp 2 phương pháp:
      - Dùng Extractive để chọn thông tin quan trọng
      - Dùng Abstractive để paraphrase và làm mượt
      - Kết quả: Vừa chính xác vừa tự nhiên
""")

print("\n" + "="*80)
print("EXPERIMENT COMPLETED! 🎉")
print("="*80)